# WINDOW FUNCTIONS

- ROW_NUMBER
- RANK
- DENSE_RANK
- FIRST_VALUE
- LAST_VALUE
- CUME_DIST
- PERCENT_RANK
- NTILE
- LEAD
- LAG
- PERCENTILE_CONT
- PERCENTILE_DESC

# OVER
## Esta clausula es requerida para todas las funciones de ventana. Utiliza los parametros PARTITION BY & ORDER BY.
---


## Consulta Principal














In [ ]:
SELECT Distinct CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName
  FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
  JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID


# ROW_NUMBER() - Genera secuencia de numeros basados en partición y orden.
## Objetivo:  Asignar ID a cada usario ordernado por Nombre y Apellido

In [ ]:
With Sellers as 
(SELECT Distinct CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName
  FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
  JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID
) Select ROW_NUMBER() OVER(Order BY SellerName) as SellerID, SellerName from Sellers

## Objectivo: Asignar ID a cada vendedor, dividido alfabeticamente por la primera letra del apellido ordernado por nombre. 

In [ ]:
With Sellers as 
(SELECT Distinct  P.FirstName, P.LastName
  FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
  JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID
) Select ROW_NUMBER() OVER(Partition By Left(LastName, 1) Order BY FirstName) as SellerID, LastName, FirstName from Sellers

## Identificar Duplicados

In [ ]:
select Product
,Price
,Category
, Row_Number() OVER(PARTITION by Product Order By Price)  as RowID from (
    Values ('Sport-100 Helmet, Red', '55.00', 'Helmets')
          ,('Sport-100 Helmet, Blue', '55.00', 'Helmets')
          ,('Sport-100 Helmet, Red', '55.00', 'Helmets')
          ,('Long-Sleeve Logo Jersey, M', '35.00', 'Jerseys')
          ,('Long-Sleeve Logo Jersey, L', '38.00', 'Jerseys')
          ,('Long-Sleeve Logo Jersey, XL', '40.00', 'Jerseys')
) as Inventory(Product,Price,Category)







# NTILE - Divide en grupos iguales
## Objectivo: Dividir el grupo de vendedores en 3 equipos.

In [ ]:
With Sellers as 
(SELECT Distinct CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName
  FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
  JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID
) Select NTILE(3) OVER(Order BY SellerName) as GroupID, SellerName from Sellers

## Objectivo: Dividir el grupo de vendedores en 3 equipos basado en ventas de mayor a menor, grupo 1 siendo el mejor en ventas.

In [ ]:
With Sellers as 
(SELECT Distinct CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName
    ,SUM(TotalDue) as SalesTotal
  FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
  JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID
  GROUP BY CONCAT_WS(' ', P.FirstName, P.LastName)
) Select NTILE(3) OVER(Order BY SalesTotal desc) as GroupID, SellerName, Concat('$',Format(SalesTotal,'#,0.00')) as SalesTotal from Sellers

# LEAD() & LAG() - Nos permite ver el registro anterior (Lag) y el proximo registro (Lead) basado en particion y sequencia. 
## Objetivo: Mostrar la diferencia entre la venta anterior y posterior a la venta X de un usuario especifico. 

In [ ]:
SELECT Distinct SalesOrderID, CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName
, Cast(OrderDate as Date) as OrderDate, TotalDue
  FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
  JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID
  where SalesPersonID = 277
  order by SalesOrderID

In [ ]:
SELECT  CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName
, Cast(OrderDate as Date) as OrderDate 
, Lag(TotalDue) Over(Order by SalesOrderID) as PreviousSell
, TotalDue
, Lead(TotalDue) Over(Order by SalesOrderID) as NextSell
  FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
  JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID
  where SalesPersonID = 277
  order by SalesOrderID

# LAST_VALUE() & FIRST_VALUE() - Retorna el primer y ultimo valor de un grupo de datos en una partición y rango.
## En adicion al uso de OVER, LAST_VALUE requiere el uso de  RANGE BETWEEN, lo cual puede utiliar los parametros UNBOUNDED PRECEEDING AND UNBOUNDED FOLLOWING.  

In [ ]:
SELECT DISTINCT CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName
    ,OrderDate, TotalDue
  FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
  JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID

In [ ]:
SELECT DISTINCT CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName
    ,FIRST_VALUE(TotalDue) OVER (PARTITION BY SalesPersonID ORDER BY SalesOrderID) AS FirstSale
    ,LAST_VALUE(TotalDue) OVER (PARTITION BY SalesPersonID ORDER BY SalesOrderID
    RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS LastSale
  FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
  JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID
  

# PERCENT_RANK - Asigna una pocision como porciento del 0 al 1 basado en la particion y orden. 

In [ ]:
With Sellers as 
(SELECT Distinct CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName
, SUM(TotalDue) as TotalSales
  FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
  JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID
  group by CONCAT_WS(' ', P.FirstName, P.LastName)
) Select SellerName, TotalSales, PERCENT_RANK() OVER(ORDER by TotalSales) from Sellers


# RANK() - Asigna un valor basado en la combinacion de particion y orden. Este valor puede ser duplicado si el rango es equitativo.

|Product | Price | Category|
|---|---|---|
|Sport-100 Helmet, Red|	55.00	|Helmets
Sport-100 Helmet, Blue |	55.00	|Helmets
Sport-100 Helmet, Green|	55.00	|Helmets
Long-Sleeve Logo Jersey, M	|35.00	|Jerseys
Long-Sleeve Logo Jersey, L	|38.00	|Jerseys
Long-Sleeve Logo Jersey, XL	|40.00	|Jerseys

In [ ]:
select Product
,Price
,Category
, Rank() OVER(ORDER BY Price DESC)  as Ranking 
 from (
    Values ('Sport-100 Helmet, Red', '55.00', 'Helmets')
          ,('Sport-100 Helmet, Blue', '55.00', 'Helmets')
          ,('Sport-100 Helmet, Green', '55.00', 'Helmets')
          ,('Long-Sleeve Logo Jersey, M', '35.00', 'Jerseys')
          ,('Long-Sleeve Logo Jersey, L', '38.00', 'Jerseys')
          ,('Long-Sleeve Logo Jersey, XL', '40.00', 'Jerseys')
) as Inventory(Product,Price,Category)

# DENSE_RANK - Similar a RANK() sin espacios entre las secuencias

In [ ]:
select Product
,Price
,Category
, Rank() OVER(ORDER BY Price DESC)  as Ranking 
, DENSE_Rank() OVER(ORDER BY Price desc)  as DenseRanking 
 from (
    Values ('Sport-100 Helmet, Red', '55.00', 'Helmets')
          ,('Sport-100 Helmet, Blue', '55.00', 'Helmets')
          ,('Sport-100 Helmet, Green', '55.00', 'Helmets')
          ,('Long-Sleeve Logo Jersey, M', '35.00', 'Jerseys')
          ,('Long-Sleeve Logo Jersey, L', '38.00', 'Jerseys')
          ,('Long-Sleeve Logo Jersey, XL', '40.00', 'Jerseys')
) as Inventory(Product,Price,Category)

# CUME_DIST() - Muestra la distribución acumulada de un grupo de valores. Basicamente muestra la pocision de un valor en relacion al valor grupal. 

In [ ]:
With Sellers as 
(SELECT  Distinct CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName
, SUM(TotalDue) as TotalSales
  FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
  JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID
  group by CONCAT_WS(' ', P.FirstName, P.LastName)
) Select SellerName, TotalSales
, CAST(CUME_DIST() OVER(ORDER by TotalSales) AS DECIMAL(8,3)) AS Distance 
from Sellers

# PERCENTILE_CONT() - Calcula el porcentaje basado en una distribución continua. Los resultados son interpolados y puede que no concuerden con un valor en el set de datos. 
## Requiere el uso de la clausula OVER y la clausula WITHIN GROUP

In [ ]:

SELECT Distinct P.BusinessEntityID, CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName, D.Name as Department, CS.CurrentSalary
FROM Person.Person P
    join HumanResources.Employee E on E.BusinessEntityID = P.BusinessEntityID
    join HumanResources.EmployeeDepartmentHistory EDH on EDH.BusinessEntityID = E.BusinessEntityID and EndDate is NULL
    join HumanResources.Department D on D.DepartmentID = EDH.DepartmentID
    CROSS APPLY
    (
        Select LAST_VALUE(Rate) OVER(ORDER BY RateChangeDate RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as CurrentSalary
        from HumanResources.EmployeePayHistory EPH
        where EPH.BusinessEntityID = P.BusinessEntityID
    ) as CS


In [ ]:
With SalaryData as 
(SELECT Distinct P.BusinessEntityID, CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName, D.Name as Department, CS.CurrentSalary
FROM Person.Person P
    join HumanResources.Employee E on E.BusinessEntityID = P.BusinessEntityID
    join HumanResources.EmployeeDepartmentHistory EDH on EDH.BusinessEntityID = E.BusinessEntityID and EndDate is NULL
    join HumanResources.Department D on D.DepartmentID = EDH.DepartmentID
    CROSS APPLY
    (
        Select LAST_VALUE(Rate) OVER(ORDER BY RateChangeDate RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as CurrentSalary
        from HumanResources.EmployeePayHistory EPH
        where EPH.BusinessEntityID = P.BusinessEntityID
    ) as CS
) Select DISTINCT Department,
PERCENTILE_CONT(.5) WITHIN GROUP(ORDER BY CurrentSalary) OVER(PARTITION BY Department) 'Median-Interpolado'
,PERCENTILE_DISC(.5) WITHIN GROUP(ORDER BY CurrentSalary) OVER(PARTITION BY Department) 'Median-Desc' from SalaryData


## Verificación

In [ ]:
With SalaryData as 
(SELECT Distinct P.BusinessEntityID, CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName, D.Name as Department, CS.CurrentSalary
FROM Person.Person P
    join HumanResources.Employee E on E.BusinessEntityID = P.BusinessEntityID
    join HumanResources.EmployeeDepartmentHistory EDH on EDH.BusinessEntityID = E.BusinessEntityID and EndDate is NULL
    join HumanResources.Department D on D.DepartmentID = EDH.DepartmentID
    CROSS APPLY
    (
        Select LAST_VALUE(Rate) OVER(ORDER BY RateChangeDate RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as CurrentSalary
        from HumanResources.EmployeePayHistory EPH
        where EPH.BusinessEntityID = P.BusinessEntityID
    ) as CS
), Valores as ( Select DISTINCT Department,
PERCENTILE_CONT(.5) WITHIN GROUP(ORDER BY CurrentSalary) OVER(PARTITION BY Department) 'Median-Interpolado'
,PERCENTILE_DISC(.5) WITHIN GROUP(ORDER BY CurrentSalary) OVER(PARTITION BY Department) 'Median-Desc' from SalaryData

)
Select Department, [Median-Interpolado], [Median-Desc], MBC.MatchedByCONT, MBD.MatchedByDESC from Valores
outer APPLY (select count(1) as MatchedByCONT from SalaryData where CurrentSalary = [Median-Interpolado] and Valores.Department = SalaryData.Department) MBC
outer APPLY (select count(1) as MatchedByDESC from SalaryData where CurrentSalary = [Median-Desc] and Valores.Department = SalaryData.Department) MBD


# Uso de OVER en funciones de para obtener valores corrientes.

In [ ]:
SELECT SalesPersonID, CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName
,SH.OrderDate , TotalDue as OrderTotal
, SUM(TotalDue) Over(Partition by SalesPersonID Order by SalesOrderID) as RunningTotalSales
, SUM(TotalDue) Over(Partition by SalesPersonID) as TotalSales
  FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
  JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID
  order by SalesPersonID, SalesOrderID


# Reporte De Ventas

In [ ]:
With
    ReportData
    as
    (
        SELECT SalesPersonID, CONCAT_WS(' ', P.FirstName, P.LastName) AS SellerName
, SH.OrderDate , TotalDue as OrderTotal
, SalesOrderID
, SUM(TotalDue) Over(Partition by SalesPersonID Order by SalesOrderID) as RunningTotalSales
, SUM(TotalDue) Over(Partition by SalesPersonID) as TotalSales
, AVG(TotalDue) Over(Partition by SalesPersonID) as AverageSalesAmount
, FIRST_VALUE(TotalDue) Over(Partition by SalesPersonID Order by SalesOrderID) AS FirstOrderTotal
, FIRST_VALUE(OrderDate) Over(Partition by SalesPersonID Order by SalesOrderID) AS FirstOrderDate
, LAST_VALUE(TotalDue) Over(Partition by SalesPersonID Order by SalesOrderID RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS LastOrderTotal
, LAST_VALUE(OrderDate) Over(Partition by SalesPersonID Order by SalesOrderID RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS LastOrderDate
        FROM [AdventureWorks2017].[Sales].[SalesOrderHeader] SH
            JOIN Person.Person P ON P.BusinessEntityID = SH.SalesPersonID
    )
Select SellerName
, OrderDate
, OrderTotal
, RunningTotalSales
, TotalSales
, AverageSalesAmount
, FirstOrderDate
, FirstOrderTotal
, LastOrderDate
, LastOrderTotal
,NTILE(3) OVER(Order BY TotalSales desc) as SalesAmountTier
from ReportData
ORDER by SalesPersonID, SalesOrderID


![Sales Report by Seller and Running Total](SalesReportByRunningValues.png)